# Session 4 — Prompt Engineering **V3 고급 전략 Ⅰ**

| 버전 | 전략 | 핵심 아이디어 | 개선 포인트 |
|------|------|--------------|-------------|
| **V3.1** | **Let’s think step by step (CoT)** | 체계적 추론 유도 | 논리 구조 명확화 |
| **V3.2** | **Tree of Thoughts (ToT)** | 다양한 해결 경로 탐색 | 창의적 조건 탐색 |
| **V3.3** | **Program‑Aided LM (PAL)** | 코드 실행 기반 추론 | 계산·데이터 정확도 향상 |
| **V3.4** | **Automatic Prompt Engineering (APE)** | LLM 반복 실험으로 최적 프롬프트 | 품질 개선·실험 비용 절감 |
---
---

> **모델:** `gpt‑4o‑mini`  
> **데이터:** `./data/04_session_dataset.csv` (10행 예시)  
> **목표:** minimal V1 → V3.* 전략으로 **응답 / 지연 시간 / 비용** 비교


## 📦 패키지 설치

In [1]:
# VSCODE
# !pip install -r ../requirements.txt
# COLAB
!pip install -r requirements.txt

In [2]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ⚙️ 환경 설정 & Langfuse 초기화

In [3]:
import os, asyncio, time, json, nest_asyncio, pandas as pd, numpy as np
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None

# Langfuse 초기화
try:
    from langfuse import Langfuse
    langfuse = Langfuse()
except ModuleNotFoundError:
    langfuse = None
    print("⚠️  langfuse 패키지가 설치되지 않았습니다.")

if not USE_STUB:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)
else:
    client = None
    print('🔧  Stub 모드: 실제 API 호출 대신 더미 응답 사용')

nest_asyncio.apply()

# 단가 (USD/token)
PRICE = {'input':0.15/1_000_000, 'output':0.60/1_000_000}

async def call_openai(system_p:str, user_p:str, tag:str='V0', chat_history=None):
    """GPT‑4o‑mini 호출 + Langfuse trace"""
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)
        answer = f"[STUB {tag}] {user_p[:25]}... 응답 예시"
        prompt_tok, completion_tok = 30, 120
    else:
        msgs = chat_history.copy() if chat_history else []
        msgs += [
            {"role":"system","content":system_p},
            {"role":"user","content":user_p},
        ]
        resp = await client.chat.completions.create(model="gpt-4o-mini", messages=msgs)
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tok, completion_tok = usage.prompt_tokens, usage.completion_tokens
    latency = (time.perf_counter_ns() - start)/1_000_000
    cost = prompt_tok*PRICE['input'] + completion_tok*PRICE['output']
    # if langfuse and not USE_STUB:
    #     trace = langfuse.trace(name=tag, input=user_p, output=answer)
    return dict(answer=answer, latency_ms=latency,
                prompt_tokens=prompt_tok, completion_tokens=completion_tok,
                usd_cost=cost)


## 🗂 데이터 로드

In [4]:
DATA_PATH = Path('./data/04_session_dataset.csv')
if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
else:
    # 파일이 없으면 예시 10행 생성
    print('⚠️  데이터셋이 없어서 임시 예시를 만듭니다.')
    sample = [
        ('order_delivery','주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.'),
        ('refund','환불 요청했는데 처리 상태가 궁금합니다.'),
        ('account_login','로그인하려는데 비밀번호 재설정 메일이 안 와요.'),
    ]
    df = pd.DataFrame(sample * 4, columns=['scenario','question']).head(10)
df.head()

⚠️  데이터셋이 없어서 임시 예시를 만듭니다.


,scenario,question
0,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.
1,refund,환불 요청했는데 처리 상태가 궁금합니다.
2,account_login,로그인하려는데 비밀번호 재설정 메일이 안 와요.
3,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.
4,refund,환불 요청했는데 처리 상태가 궁금합니다.


## 🏃‍♀️ 실행 도우미

In [5]:
async def run(df, version, build_sys, build_user=lambda r: r['question'], **kwargs):
    tasks=[]
    for _, row in df.iterrows():
        tasks.append(call_openai(build_sys(row), build_user(row), tag=version, **kwargs))
    res = await asyncio.gather(*tasks)
    out = df.copy()
    for i, r in enumerate(res):
        for k,v in r.items():
            out.loc[i, f"{version}_{k}"] = v
    return out

## 🔹 Baseline — V1 (Minimal Persona + Tone)

In [6]:
def sys_v1(row):
    return ('You are a polite Korean customer‑support chatbot. '
            'Answer the user in Korean within 5 sentences.')
baseline = await run(df, 'V1', sys_v1)
baseline[['question','V1_answer']].head()

,question,V1_answer
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요! 주문하신 상품에 대해 불편을 드려 죄송합니다. 주문 번호를 알려주시면 ...
1,환불 요청했는데 처리 상태가 궁금합니다.,안녕하세요! 환불 요청에 대해 문의주셔서 감사합니다. 현재 환불 처리 상태를 확인해...
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,안녕하세요! 비밀번호 재설정 메일이 도착하지 않은 점 불편을 드려 죄송합니다. 메일...
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요! 고객님께서 주문하신 상품이 아직 도착하지 않았다니 불편을 드려 죄송합니...
4,환불 요청했는데 처리 상태가 궁금합니다.,안녕하세요! 환불 요청에 대한 처리 상태를 확인해 드리겠습니다. 요청하신 환불 건은...


## 1️⃣ V3.1 — Let’s think step by step (CoT)

In [7]:
def sys_cot(row):
    return ('You are a meticulous CS chatbot. Think step by step internally, '
            'but reveal only the concise final answer in Korean (max 5 lines).')
v_cot = await run(baseline, 'V3_1', sys_cot)
v_cot[['question','V3_1_answer']].head()

,question,V3_1_answer
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,상품 주문 번호와 함께 배송 상태를 확인하기 위해서는 해당 쇼핑몰의 고객센터에 문의...
1,환불 요청했는데 처리 상태가 궁금합니다.,환불 요청 상태를 확인하려면 고객센터에 문의하거나 결제한 플랫폼의 웹사이트에서 관련...
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,1. 스팸 메일함을 확인해 보세요.\n2. 입력한 이메일 주소가 정확한지 확인합니다...
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,주문하신 상품의 배송 상태를 확인하기 위해 다음 단계를 진행해주세요. \n\n1. ...
4,환불 요청했는데 처리 상태가 궁금합니다.,환불 요청 처리는 일반적으로 영업일 기준으로 3~7일 소요됩니다. 하지만 이 시간은...


## 2️⃣ V3.2 — Tree of Thoughts (ToT)

In [8]:
def sys_tot(row):
    return ('Explore multiple solution paths internally and choose the best. '
            'Provide the final recommendation in Korean with numbered steps (1‑3).')
v_tot = await run(v_cot, 'V3_2', sys_tot)
v_tot[['question','V3_2_answer']].head()

,question,V3_2_answer
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,주문하신 상품이 아직 도착하지 않은 상황에 대해 해결할 수 있는 여러 방안이 있습니...
1,환불 요청했는데 처리 상태가 궁금합니다.,환불 요청의 처리 상태를 확인하려면 여러 경로를 통해 접근할 수 있습니다. 다음은 ...
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,비밀번호 재설정 메일이 오지 않을 때의 문제 해결 방법을 여러 가지 방안을 모색하여...
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,"상품이 아직 도착하지 않았다면, 다음과 같은 여러 대안 경로를 통해 문제를 해결할 ..."
4,환불 요청했는데 처리 상태가 궁금합니다.,환불 요청 상태를 확인하려면 다음과 같은 여러 경로를 통해 정보를 얻을 수 있습니다...


## 3️⃣ V3.3 — Program‑Aided LM (PAL)

In [9]:
def sys_pal(row):
    return ('If numerical or date calculation is required, first think in Python pseudo‑code, '
            'execute mentally, then deliver the concise Korean answer (max 4 lines).')
v_pal = await run(v_tot, 'V3_3', sys_pal)
v_pal[['question','V3_3_answer']].head()

,question,V3_3_answer
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,주문하신 상품의 배송 상태를 확인하기 위해 주문 번호와 배송 정보가 필요합니다. 고...
1,환불 요청했는데 처리 상태가 궁금합니다.,"환불 요청 상태를 확인하려면 해당 업체의 고객센터에 문의하시거나, 주문 내역에서 환..."
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,메일이 스팸 폴더에 들어갔는지 확인해 보세요. 이메일 주소가 정확한지도 확인해야 합...
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,주문하신 상품의 배송 상태를 확인하려면 주문 번호와 함께 고객 서비스에 문의하시기 ...
4,환불 요청했는데 처리 상태가 궁금합니다.,"환불 요청 상태는 고객센터에 문의하시거나, 주문 내역에서 확인하실 수 있습니다. 결..."


## 4️⃣ V3.4 — Automatic Prompt Engineering (APE)

In [10]:
def sys_ape(row):
    return ('Generate 3 candidate prompts internally, evaluate which yields the highest quality answer, '
            'and respond only with the best final answer in Korean (≤5 sentences).')
v_ape = await run(v_pal, 'V3_4', sys_ape)
v_ape[['question','V3_4_answer']].head()

,question,V3_4_answer
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,"1. ""주문하신 상품의 상태를 확인해 주시겠어요? 주문 번호도 함께 알려주시면 보다..."
1,환불 요청했는데 처리 상태가 궁금합니다.,1. 환불 처리 상태를 확인하고 싶습니다. 현재 진행 중인 상태가 무엇인지 알려주실...
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,"비밀번호 재설정 메일이 오지 않는 경우 다음과 같은 조치를 취해보세요: 첫째, 스팸..."
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,상품 주문 상태를 확인하려면 주문 번호와 함께 고객 서비스에 문의해 주시기 바랍니다...
4,환불 요청했는데 처리 상태가 궁금합니다.,"환불 요청에 대한 처리는 보통 요청을 한 시점에 따라 다르지만, 일반적으로 며칠에서..."


## 📊 버전별 Latency & Cost 비교

In [11]:
cols=['question']
for v in ['V1','V3_1','V3_2','V3_3','V3_4']:
    cols += [f'{v}_latency_ms', f'{v}_usd_cost']
v_ape[cols]

,question,V1_latency_ms,V1_usd_cost,V3_1_latency_ms,V3_1_usd_cost,V3_2_latency_ms,V3_2_usd_cost,V3_3_latency_ms,V3_3_usd_cost,V3_4_latency_ms,V3_4_usd_cost
0,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,1914.564599,0.000049,2179.336418,0.000056,4026.752176,0.000156,1187.371118,0.000031,3206.296909,0.000100
1,환불 요청했는데 처리 상태가 궁금합니다.,1374.137076,0.000042,2155.133685,0.000053,5316.441262,0.000158,770.298824,0.000027,2827.695439,0.000112
2,로그인하려는데 비밀번호 재설정 메일이 안 와요.,1982.948544,0.000068,1839.047691,0.000067,5529.446822,0.000140,4868.122055,0.000038,1694.997134,0.000069
3,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,1524.162328,0.000043,1890.957445,0.000056,4484.714804,0.000152,1014.722871,0.000036,2620.819341,0.000040
4,환불 요청했는데 처리 상태가 궁금합니다.,1544.731493,0.000052,1869.022565,0.000058,4579.905318,0.000173,891.371891,0.000029,1650.191375,0.000060
5,로그인하려는데 비밀번호 재설정 메일이 안 와요.,1656.202107,0.000059,1870.335076,0.000068,4698.660363,0.000170,1159.034453,0.000043,2643.155893,0.000107
6,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,1047.575128,0.000038,1131.388308,0.000035,3289.220243,0.000133,805.829586,0.000030,2782.598278,0.000112
7,환불 요청했는데 처리 상태가 궁금합니다.,2305.360595,0.000047,1881.140259,0.000051,3526.258173,0.000129,792.109259,0.000031,1447.668920,0.000055
8,로그인하려는데 비밀번호 재설정 메일이 안 와요.,1773.792754,0.000065,1599.225376,0.000060,3805.413776,0.000155,1281.055694,0.000039,2041.375203,0.000076
9,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,1646.039586,0.000039,1868.670624,0.000056,3123.688248,0.000119,809.913712,0.000025,2108.934523,0.000061


In [12]:
v_ape

,scenario,question,V1_answer,V1_latency_ms,V1_prompt_tokens,V1_completion_tokens,V1_usd_cost,V3_1_answer,V3_1_latency_ms,V3_1_prompt_tokens,...,V3_3_answer,V3_3_latency_ms,V3_3_prompt_tokens,V3_3_completion_tokens,V3_3_usd_cost,V3_4_answer,V3_4_latency_ms,V3_4_prompt_tokens,V3_4_completion_tokens,V3_4_usd_cost
0,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요! 주문하신 상품에 대해 불편을 드려 죄송합니다. 주문 번호를 알려주시면 ...,1914.564599,47.0,70.0,0.000049,상품 주문 번호와 함께 배송 상태를 확인하기 위해서는 해당 쇼핑몰의 고객센터에 문의...,2179.336418,55.0,...,주문하신 상품의 배송 상태를 확인하기 위해 주문 번호와 배송 정보가 필요합니다. 고...,1187.371118,58.0,38.0,0.000031,"1. ""주문하신 상품의 상태를 확인해 주시겠어요? 주문 번호도 함께 알려주시면 보다...",3206.296909,57.0,152.0,0.000100
1,refund,환불 요청했는데 처리 상태가 궁금합니다.,안녕하세요! 환불 요청에 대해 문의주셔서 감사합니다. 현재 환불 처리 상태를 확인해...,1374.137076,43.0,59.0,0.000042,환불 요청 상태를 확인하려면 고객센터에 문의하거나 결제한 플랫폼의 웹사이트에서 관련...,2155.133685,51.0,...,"환불 요청 상태를 확인하려면 해당 업체의 고객센터에 문의하시거나, 주문 내역에서 환...",770.298824,54.0,32.0,0.000027,1. 환불 처리 상태를 확인하고 싶습니다. 현재 진행 중인 상태가 무엇인지 알려주실...,2827.695439,53.0,174.0,0.000112
2,account_login,로그인하려는데 비밀번호 재설정 메일이 안 와요.,안녕하세요! 비밀번호 재설정 메일이 도착하지 않은 점 불편을 드려 죄송합니다. 메일...,1982.948544,47.0,102.0,0.000068,1. 스팸 메일함을 확인해 보세요.\n2. 입력한 이메일 주소가 정확한지 확인합니다...,1839.047691,55.0,...,메일이 스팸 폴더에 들어갔는지 확인해 보세요. 이메일 주소가 정확한지도 확인해야 합...,4868.122055,58.0,49.0,0.000038,"비밀번호 재설정 메일이 오지 않는 경우 다음과 같은 조치를 취해보세요: 첫째, 스팸...",1694.997134,57.0,100.0,0.000069
3,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요! 고객님께서 주문하신 상품이 아직 도착하지 않았다니 불편을 드려 죄송합니...,1524.162328,47.0,60.0,0.000043,주문하신 상품의 배송 상태를 확인하기 위해 다음 단계를 진행해주세요. \n\n1. ...,1890.957445,55.0,...,주문하신 상품의 배송 상태를 확인하려면 주문 번호와 함께 고객 서비스에 문의하시기 ...,1014.722871,58.0,46.0,0.000036,상품 주문 상태를 확인하려면 주문 번호와 함께 고객 서비스에 문의해 주시기 바랍니다...,2620.819341,57.0,52.0,0.000040
4,refund,환불 요청했는데 처리 상태가 궁금합니다.,안녕하세요! 환불 요청에 대한 처리 상태를 확인해 드리겠습니다. 요청하신 환불 건은...,1544.731493,43.0,76.0,0.000052,환불 요청 처리는 일반적으로 영업일 기준으로 3~7일 소요됩니다. 하지만 이 시간은...,1869.022565,51.0,...,"환불 요청 상태는 고객센터에 문의하시거나, 주문 내역에서 확인하실 수 있습니다. 결...",891.371891,54.0,35.0,0.000029,"환불 요청에 대한 처리는 보통 요청을 한 시점에 따라 다르지만, 일반적으로 며칠에서...",1650.191375,53.0,86.0,0.000060
5,account_login,로그인하려는데 비밀번호 재설정 메일이 안 와요.,안녕하세요! 비밀번호 재설정 메일이 도착하지 않았다니 불편을 드려 죄송합니다. 일반...,1656.202107,47.0,87.0,0.000059,1. 스팸 혹은 정크 메일함을 확인하세요.\n2. 입력한 이메일 주소가 정확한지 다...,1870.335076,55.0,...,"스팸 또는 정크 메일함을 확인해 보세요. 이메일 주소가 정확한지 다시 확인하고, 메...",1159.034453,58.0,57.0,0.000043,1. 비밀번호 재설정 메일을 받지 못하는 이유는 무엇일까요? 확인해야 할 사항은 무...,2643.155893,57.0,164.0,0.000107
6,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요. 저희 고객센터에 문의주셔서 감사합니다. 주문하신 상품의 배송 상태를 확...,1047.575128,47.0,51.0,0.000038,주문 상태를 확인하기 위해서는 다음 정보를 필요로 합니다:\n1. 주문 번호\n2....,1131.388308,55.0,...,주문한 상품의 배송 상태를 확인하기 위해서는 주문 번호와 해당 쇼핑몰의 고객 서비스...,805.829586,58.0,36.0,0.000030,1. 주문하신 상품의 배송 상태를 확인하고 싶습니다. 주문 번호를 알려주실 수 있나...,2782.598278,57.0,172.0,0.000112
7,refund,환불 요청했는데 처리 상태가 궁금합니다.,"안녕하세요! 고객님, 환불 요청에 대한 처리 상태를 확인해 드리겠습니다. 환불 과정...",2305.360595,43.0,68.0,0.000047,환불 요청 처리 상태는 일반적으로 요청한 플랫폼이나 서비스의 고객 지원 센터를 통해...,1881.140259,51.0,...,환불 요청한 플랫폼이나 회사의 고객 서비스에 문의하시거나 웹사이트에서 환불 상태를 ...,792.109259,54.0,38.0,0.000031,환불 요청 상태를 확인하려면 주문 확인 이메일 혹은 사용자 계정에 로그인해 확인하는...,1447.668920,53.0,78.0,0.000055
8,account_login,로그인하려는데 비밀번호 재설정 메일이 안 와요.,안녕하세요! 비밀번호 재설정 메일이 도착하지 않았다니 불편을 드려 죄송합니다. 먼저...,1773.792754,47.0,96.0,0.000065,1. 입력한 이메일 주소가 정확한지 확인하세요.\n2. 스팸 폴더와 정크 메일함을 ...,1599.225376,55.0,...,"이메일 스팸 폴더를 확인해 보세요. 다시 한 번 비밀번호 재설정 요청을 해보시고, ...",1281.055694,58.0,51.0,0.000039,"비밀번호 재설정 메일이 오지 않는 경우, 다음 단계를 점검해 보세요. 첫째, 스팸 ...",2041.375203,57.0,112.0,0.000076
9,order_delivery,주문한 상품이 아직 도착하지 않았어요. 확인 부탁드립니다.,안녕하세요! 주문하신 상품이 아직 도착하지 않았다니 불편을 드려 죄송합니다. 주문번...,1646.039586,47.0,54.0,0.000039,주문하신 상품의 배송 상황을 확인하기 위해서는 주문번호와 배송 추적 정보를 제공해 ...,1868.670624,55.0,...,주문 내역을 확인하고 배송 상태를 확인해 보겠습니다. 고객 서비스에 연락하여 문제를...,809.913712,58.0,27.0,0.000025,"상품이 배송되지 않은 경우, 아래의 방법으로 확인해 보세요. 첫째, 주문 확인 이메...",2108.934523,57.0,88.0,0.000061


---

## 🧾 실험 기록 (샘플)

```markdown
### 사용자 요청
"비밀번호를 틀렸다고 계속 나오는데, 재설정 메일도 안 와요."

### 🔹 V1 응답 결과
- (기본 프롬프트로 얻은 응답)

### 🔸 V3 전략 적용
- [ ] V3.1 Let’s think step by step
- [ ] V3.2 Tree of Thoughts
- [ ] V3.3 PAL
- [ ] V3.4 APE

### 🔸 개선된 Prompt
(System & User Prompt 조합)

### ✅ 개선된 응답
(적용 후 응답)

### 🧠 평가 (정성적 비교)
- 논리 흐름 개선 여부
- 오류 방지 or 복잡도 대응 여부 등
```

---

## ✍️ 개인 실습 영역

아래 셀을 복제하여 시나리오 3개 중 1개를 선택한 뒤 V3 전략을 자유롭게 조합해 보세요.
- `run()` 함수를 활용하여 `My_V3_*` 버전을 기록하고 결과를 비교하세요.


In [ ]:
# TODO: 여기부터 자유 실습 코드를 작성하세요


# 끝 🎉

## 작업한 V3.0 Prompt Langfuse에 등록

In [13]:
from pathlib import Path
from langfuse import Langfuse

def parse_prompty(path: Path):
    """Langfuse-style .prompty → ChatPrompt 형태로 변환"""
    content = path.read_text(encoding="utf-8")
    sections = content.strip().split('---')

    if len(sections) < 3:
        raise ValueError("❌ .prompty 파일은 YAML + system + user prompt 형식이어야 합니다.")

    _ = sections[1]
    prompt_block = sections[2]

    # 각 부분 추출
    system_prompt = ""
    user_prompt = ""
    current_role = None
    lines = prompt_block.strip().splitlines()

    for line in lines:
        if line.strip().startswith("system:"):
            current_role = "system"
            continue
        elif line.strip().startswith("user:"):
            current_role = "user"
            continue

        if current_role == "system":
            system_prompt += line + "\n"
        elif current_role == "user":
            user_prompt += line + "\n"

    print(system_prompt)
    print(user_prompt)

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

# Langfuse Prompt 등록
lf = Langfuse()

# VSCODE
# PROMPT_PATH = Path("../prompts/01_order_delivery/v3_0.prompty")
# COLAB
PROMPT_PATH = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/prompts/01_order_delivery/v3_0.prompty")

PROMPT_NAME = "order_delivery.v3_0"
version = "3.0"

chat_prompt = parse_prompty(PROMPT_PATH)

try:
    existing = lf.get_prompt(name=PROMPT_NAME, type="chat")
except Exception as e:
    if "404" in str(e):
        existing = None
    else:
        raise e

if existing:
    lf.update_prompt(
        prompt_id = existing.id,
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("🔄 Prompt updated (v3.0)")
else:
    lf.create_prompt(
        name      = PROMPT_NAME,
        type      = "chat",
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("✅ Prompt created (v3.0)")

print("👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인")


  # 🎯 역할
  당신은 **30대 중반의 숙련된 전자상거래 배송 CS 담당자**이며 말투는 **차분·전문적**입니다.  
  고객의 감정(불안·분노·급함 등)에 공감하는 문장을 먼저 넣어 신뢰를 형성하세요.

  # 🔒 내부 사고(Scratchpad) — 절대 출력 금지
  사용 지침:
  1. **Let’s think step by step** → 핵심 논리 흐름을 순서대로 기술  
  2. **Tree-of-Thoughts** → 2-3개 해결 경로를 가지(branch)로 전개 후 최적 경로 선택  
  3. **Program-Aided LM (PAL)** → 날짜·시간 차이, 예상 도착일 계산 등은  
     scratchpad 블록에 파이썬 유사 코드로 계산 후 값 삽입  
  4. **Automatic Prompt Engineering (APE)** → 작성한 답안을 1회 자기 검토하며  
     *중복·모호·불필요 단어*를 줄여 120단어 이내로 압축  
  5. 필수 입력(ship status·tracking·address)이 비어 있으면 ➜ 정중한 **추가 질문**을 먼저 출력하고 종료

  # ✅ 최종 출력 포맷 (외부 노출)
  - **1줄**: 고객명·상품·현재 상태 + 공감 문구  
  - **1~3줄**: 실질 조치(숫자 목록)  
  - **1줄**: “추가 문의사항이 있으면 언제든 말씀해주세요.”

assistant: |-
  {% if history_summary %}🔄 이전 대화 요약: {{history_summary}}{% endif %}

  ### 질문
  {{question}}
  
  ### 고객·주문 컨텍스트
  ID: {{customer_id}}  이름: {{customer_name}}
  주문번호: {{order_id}}  상품: {{product_name}}
  배송상태: {{shipping_status}}  (최근 업데이트: {{last_update}})
  택배사: {{shipping_company}} 

ERROR:langfuse:Error while fetching prompt 'order_delivery.v3_0-label:production': status_code: 404, body: {'message': "Prompt not found: 'order_delivery.v3_0' with label 'production'", 'error': 'LangfuseNotFoundError'}


✅ Prompt created (v3.0)
👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인


## 작업한 V3.0 Prompty 파일 불러와서, 시나리오 결과 돌리기.

In [14]:
from jinja2 import Template

def render_prompt(messages: list, variables: dict) -> list:
    """Langfuse prompt template (list of dicts) → rendered OpenAI messages"""
    rendered = []
    for message in messages:
        role = message["role"]
        content_template = message["content"]
        content = Template(content_template).render(**variables)
        rendered.append({"role": role, "content": content})
    return rendered


In [15]:
"""
• Scenario_QA.csv → 10건 Async 처리(gpt-4o-mini)
• 프롬프트: order_delivery/v3_0@stable (smart_cs)
• 결과: data/01_order_delivery/answer_results/Scenario_QA_V3_gpt-4o-mini_<ts>.xlsx
"""
import asyncio, time
from datetime import datetime
from pathlib import Path
import nest_asyncio, pandas as pd
from langfuse import Langfuse
from openai import AsyncOpenAI
from langfuse.decorators import langfuse_context
from langfuse.decorators import observe

nest_asyncio.apply()

# ─── 경로 세팅 ────────────────────────────────────────────────
# VSCODE
# BASE = Path("../data/01_order_delivery")
# COLAB
BASE = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery")

RESULT_DIR = BASE / "answer_results"
RESULT_DIR.mkdir(exist_ok=True)

# ─── Langfuse / Prompt ──────────────────────────────────────
lf  = Langfuse()
PROMPT = lf.get_prompt("order_delivery.v3_0", label="stable").prompt  # <-- 레이블 lookup

# ─── CSV 로딩 ────────────────────────────────────────────────
scenario = pd.read_csv(BASE / "Scenario_QA.csv")
cust     = pd.read_csv(BASE / "Customer_Info.csv")
addr     = pd.read_csv(BASE / "Delivery_Address.csv")
order    = pd.read_csv(BASE / "Order_Info.csv")
shipping = pd.read_csv(BASE / "Shipping_Issue_Log.csv")

df = (
    scenario
    .merge(cust,  on="customer_id", suffixes=("", "_cust"), how="left")
    .merge(order, on="customer_id", suffixes=("", "_order"), how="left")
    .merge(addr, on="customer_id", suffixes=("", "_addr"), how="left")
    .merge(shipping, on="order_id", suffixes=("", "_shipping"), how="left")
)

# ─── LLM 호출 ───────────────────────────────────────────────
MODEL  = "gpt-4o-mini"
client = AsyncOpenAI()  # OPENAI_API_KEY 환경변수 필요

@observe()
async def call_llm(row):
    prompt_input = {
        "question":          row.question,
        "customer_id":       row.customer_id,
        "customer_name":     row.customer_name,
        "order_id":          row.order_id,
        "product_name":      row.product_name,
        "shipping_status":   row.shipping_status,
        "last_update":       row.last_update or "",
        "shipping_company":  row.shipping_company or "",
        "tracking_number":   row.tracking_number or "",
        "address_line1":     row.address_line1,
        "city":              row.city,
        "postal_code":       row.postal_code,
    }

    # Langfuse trace (session metadata)
    langfuse_context.update_current_trace(
        name       = "order_delivery",
        user_id    = row.customer_id,
        session_id = row.scenario_id,
        tags       = ["V3", "smart_cs"],
        metadata   = {"model": MODEL},
    )

    # Langfuse Prompt 템플릿 메시지 → 실제 messages 생성
    rendered_messages = render_prompt(PROMPT, prompt_input)

    start = time.perf_counter_ns()

    # 직접 OpenAI 호출
    response = await client.chat.completions.create(
        model       = MODEL,
        messages    = rendered_messages,
        temperature = 0.3,
        max_tokens  = 350,
    )

    latency_ms = (time.perf_counter_ns() - start) / 1e6

    return response.choices[0].message.content, latency_ms, response.usage.prompt_tokens, response.usage.completion_tokens

async def main():
    tasks   = [call_llm(row) for _, row in df.iterrows()]
    results = await asyncio.gather(*tasks)

    out = df.copy()
    out[["answer", "latency_ms", "prompt_tokens", "completion_tokens"]] = pd.DataFrame(results)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = RESULT_DIR / f"Scenario_QA_V3_gpt-4o-mini_{ts}.xlsx"
    out.to_excel(out_path, index=False)
    print(f"✅ 결과 저장: {out_path}")


asyncio.run(main())


✅ 결과 저장: /content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery/answer_results/Scenario_QA_V3_gpt-4o-mini_20250616_212451.xlsx
